<a href="https://colab.research.google.com/github/sjslack18/DH100-Project/blob/main/DH100_Notebook_2_(Processing).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
path = "/content/drive/MyDrive/Sam Slack/scraped_data.csv"

from nltk.tokenize import word_tokenize
import nltk 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

  
df = pd.read_csv(path)
df.head(5)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,title,text,date
0,Press Briefing by White House COVID-19 Respons...,About Search Via Teleconference 10:19 A.M. ED...,2021-06-08 10:19:00
1,Press Briefing by Press Secretary Jen Psaki an...,About Search White House Press Secretary Jame...,2021-06-07 12:42:00
2,Press Briefing by Press Secretary Jen Psaki an...,About Search White House Press Secretary Jame...,2021-06-04 13:39:00
3,Press Briefing by Press Secretary Jen Psaki | ...,About Search White House Press Secretary Jame...,2021-06-03 12:56:00
4,Press Briefing by White House COVID-19 Respons...,About Search Via Teleconference 11:10 A.M. ED...,2021-06-03 11:10:00


Data scraped from the APP website (separate notebook) is downloaded from drive and loaded into a new Python notebook (for easier NLP uses). 

In [34]:
df["text"] = df["text"].apply(str.lower)

Convert all text to lowercase to help standardize before tokenization.

In [35]:
df["tokenized_text"] = df["text"].apply(nltk.word_tokenize)
df.head(5)

,title,text,date,tokenized_text
0,Press Briefing by White House COVID-19 Respons...,about search via teleconference 10:19 a.m. ed...,2021-06-08 10:19:00,"[about, search, via, teleconference, 10:19, a...."
1,Press Briefing by Press Secretary Jen Psaki an...,about search white house press secretary jame...,2021-06-07 12:42:00,"[about, search, white, house, press, secretary..."
2,Press Briefing by Press Secretary Jen Psaki an...,about search white house press secretary jame...,2021-06-04 13:39:00,"[about, search, white, house, press, secretary..."
3,Press Briefing by Press Secretary Jen Psaki | ...,about search white house press secretary jame...,2021-06-03 12:56:00,"[about, search, white, house, press, secretary..."
4,Press Briefing by White House COVID-19 Respons...,about search via teleconference 11:10 a.m. ed...,2021-06-03 11:10:00,"[about, search, via, teleconference, 11:10, a...."


Tokenize text with NLTK word_tokenize.


In [36]:
lemmatizer = WordNetLemmatizer()
def lemmatize_list(list):
  lemmatized_list = [lemmatizer.lemmatize(word) for word in list]
  return lemmatized_list
#test the lemmatizer
test = df['tokenized_text'][1]
lemmatize_list(test)[1:10]

#apply to tokenized words
df['tokenized_text'] = df['tokenized_text'].apply(lemmatize_list)
df.head(5)

,title,text,date,tokenized_text
0,Press Briefing by White House COVID-19 Respons...,about search via teleconference 10:19 a.m. ed...,2021-06-08 10:19:00,"[about, search, via, teleconference, 10:19, a...."
1,Press Briefing by Press Secretary Jen Psaki an...,about search white house press secretary jame...,2021-06-07 12:42:00,"[about, search, white, house, press, secretary..."
2,Press Briefing by Press Secretary Jen Psaki an...,about search white house press secretary jame...,2021-06-04 13:39:00,"[about, search, white, house, press, secretary..."
3,Press Briefing by Press Secretary Jen Psaki | ...,about search white house press secretary jame...,2021-06-03 12:56:00,"[about, search, white, house, press, secretary..."
4,Press Briefing by White House COVID-19 Respons...,about search via teleconference 11:10 a.m. ed...,2021-06-03 11:10:00,"[about, search, via, teleconference, 11:10, a...."


Create a lemmatizer that lemmatizes (finds the root of each word for standardization) by the row.

In [37]:
environment = ["environment", "green", "climate", "atmosphere","carbon", "emission","clean","solar","wind","fuel","epa"]
defense = ["security","conflict","weapon","peace","missile","threat","military","aggression","defense","military"]
economy= ["economy","job","market","stimulus","unemployment","wage","interest","stock"]
public_health = ["virus","health","safety","pandemic","epidemic","COVID-19","opioid","drug"]
education = ["school","curriculum","teach","teacher","education","degree","student","learn"]

Create word categories that have target words to search for. This is done multiple times as an exploratory process to find the best words that will show something about trends in the data.

In [38]:
def cat_count(lst,cat_lst):
  return sum(x in cat_lst for x in lst)

test = df['tokenized_text'].apply(lambda lst: cat_count(lst,environment))

df["environment_count"] = df['tokenized_text'].apply(lambda lst: cat_count(lst,environment))
df["defense_count"] = df['tokenized_text'].apply(lambda lst: cat_count(lst,defense))
df["economy_count"] = df['tokenized_text'].apply(lambda lst: cat_count(lst,economy))
df["public_health_count"] = df['tokenized_text'].apply(lambda lst: cat_count(lst,public_health))
df["education_count"] = df['tokenized_text'].apply(lambda lst: cat_count(lst,education))
df.head(5)

,title,text,date,tokenized_text,environment_count,defense_count,economy_count,public_health_count,education_count
0,Press Briefing by White House COVID-19 Respons...,about search via teleconference 10:19 a.m. ed...,2021-06-08 10:19:00,"[about, search, via, teleconference, 10:19, a....",0,3,4,14,4
1,Press Briefing by Press Secretary Jen Psaki an...,about search white house press secretary jame...,2021-06-07 12:42:00,"[about, search, white, house, press, secretary...",14,26,21,21,1
2,Press Briefing by Press Secretary Jen Psaki an...,about search white house press secretary jame...,2021-06-04 13:39:00,"[about, search, white, house, press, secretary...",3,13,84,14,11
3,Press Briefing by Press Secretary Jen Psaki | ...,about search white house press secretary jame...,2021-06-03 12:56:00,"[about, search, white, house, press, secretary...",1,18,35,22,2
4,Press Briefing by White House COVID-19 Respons...,about search via teleconference 11:10 a.m. ed...,2021-06-03 11:10:00,"[about, search, via, teleconference, 11:10, a....",0,6,1,35,1


Create counts of frequencies of target words for each briefing.

In [39]:
 df['environment_z'] = stats.zscore(df['environment_count'])
df['defense_z'] = stats.zscore(df['defense_count'])
df['economy_z']= stats.zscore(df['economy_count'])
df['public_health_z'] = stats.zscore(df['public_health_count'])
df['education_z'] = stats.zscore(df['education_count'])
df.head(10)


,title,text,date,tokenized_text,environment_count,defense_count,economy_count,public_health_count,education_count,environment_z,defense_z,economy_z,public_health_z,education_z
0,Press Briefing by White House COVID-19 Respons...,about search via teleconference 10:19 a.m. ed...,2021-06-08 10:19:00,"[about, search, via, teleconference, 10:19, a....",0,3,4,14,4,-0.407324,-0.713793,-0.566748,0.923794,0.066124
1,Press Briefing by Press Secretary Jen Psaki an...,about search white house press secretary jame...,2021-06-07 12:42:00,"[about, search, white, house, press, secretary...",14,26,21,21,1,1.827429,0.474526,0.487367,1.644607,-0.264196
2,Press Briefing by Press Secretary Jen Psaki an...,about search white house press secretary jame...,2021-06-04 13:39:00,"[about, search, white, house, press, secretary...",3,13,84,14,11,0.071551,-0.197133,4.393796,0.923794,0.836869
3,Press Briefing by Press Secretary Jen Psaki | ...,about search white house press secretary jame...,2021-06-03 12:56:00,"[about, search, white, house, press, secretary...",1,18,35,22,2,-0.247699,0.061197,1.355463,1.747580,-0.154089
4,Press Briefing by White House COVID-19 Respons...,about search via teleconference 11:10 a.m. ed...,2021-06-03 11:10:00,"[about, search, via, teleconference, 11:10, a....",0,6,1,35,1,-0.407324,-0.558795,-0.752769,3.086233,-0.264196
5,Press Briefing by Press Secretary Jen Psaki | ...,about search white house press secretary jame...,2021-06-02 12:27:00,"[about, search, white, house, press, secretary...",1,17,16,11,13,-0.247699,0.009531,0.177333,0.614874,1.057082
6,Press Gaggle by Principal Deputy Press Secreta...,about search aboard air force oneen route tul...,2021-06-01 12:42:00,"[about, search, aboard, air, force, oneen, rou...",0,5,7,1,15,-0.407324,-0.610461,-0.380728,-0.414859,1.277295
7,Press Gaggle by Principal Deputy Press Secreta...,about search aboard air force oneen route ham...,2021-05-28 13:24:00,"[about, search, aboard, air, force, oneen, rou...",0,3,8,3,1,-0.407324,-0.713793,-0.318721,-0.208912,-0.264196
8,Press Gaggle by Press Secretary Jen Psaki | Th...,about search white house press secretary aboa...,2021-05-27 12:39:00,"[about, search, white, house, press, secretary...",1,1,15,11,0,-0.247699,-0.817125,0.115327,0.614874,-0.374302
9,Press Briefing by Principal Deputy Press Secre...,about search james s. brady press briefing ro...,2021-05-26 13:29:00,"[about, search, james, s., brady, press, brief...",4,1,20,32,4,0.231177,-0.817125,0.425361,2.777313,0.066124


Standardize counts into zscores to better show progression of category over time, since absolute counts don't mean much.

In [40]:
from datetime import datetime
df['date']= df['date'].apply(lambda x: datetime.fromisoformat(x))

In [41]:
min(df['date'])

Timestamp('1993-01-27 00:00:00')

In [42]:
def administration_sorter(date):
  if date < pd.Timestamp(2001,1,20,0,0,0):
    return "Clinton"
  elif date < pd.Timestamp(2009,1,20,0,0,0):
    return "Bush"
  elif date < pd.Timestamp(2017,1,20,0,0,0):
    return "Obama"
  elif date < pd.Timestamp(2021,1,20,0,0,0):
    return "Trump"
  else:
    return "Biden"

In [43]:
df['admin'] = df['date'].apply(administration_sorter)

In [44]:
avg_counts_by_admin = df.groupby('admin')['environment_count','defense_count','economy_count','public_health_count','education_count'].mean()
avg_counts_by_admin

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,environment_count,defense_count,economy_count,public_health_count,education_count
admin,,,,,
Biden,4.941176,10.595588,14.852941,19.801471,12.911765
Bush,2.198081,17.579007,9.406885,2.957675,2.020880
Clinton,1.859343,10.824949,10.089322,4.446099,3.756160
Obama,3.569275,24.064928,20.821449,6.597681,3.678841
Trump,2.226708,12.645963,10.267081,5.307453,3.313665


In [45]:
avg_z_by_admin = df.groupby('admin')['environment_z','defense_z','economy_z','public_health_z','education_z'].mean()
avg_z_by_admin

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,environment_z,defense_z,economy_z,public_health_z,education_z
admin,,,,,
Biden,0.381412,-0.321359,0.106208,1.521191,1.047367
Bush,-0.056455,0.039446,-0.231485,-0.213270,-0.151790
Clinton,-0.110526,-0.309509,-0.189169,-0.060003,0.039275
Obama,0.162422,0.374548,0.476296,0.161553,0.030762
Trump,-0.051885,-0.215424,-0.178147,0.028694,-0.009446


In [46]:
import plotly.graph_objects as go
#categories must also loop around to close the plotly radar graph
categories = ["Environment","Defense","Economy","Public Health","Education","Environment"]
zradar = go.Figure()
biden_z = avg_z_by_admin.loc["Biden"].values.tolist()
bush_z = avg_z_by_admin.loc["Bush"].values.tolist()
clinton_z = avg_z_by_admin.loc["Clinton"].values.tolist()
obama_z = avg_z_by_admin.loc["Obama"].values.tolist()
trump_z = avg_z_by_admin.loc["Trump"].values.tolist()

#because of plotly weirdness, must repeat first value of each set to close the graph
biden_z.append(biden_z[0])
bush_z.append(bush_z[0])
clinton_z.append(clinton_z[0])
obama_z.append(obama_z[0])
trump_z.append(trump_z[0])

#radar traces
zradar.add_trace(go.Scatterpolar(r = biden_z,theta= categories,fill = 'toself',name = "Biden"))
zradar.add_trace(go.Scatterpolar(r = bush_z,theta= categories,fill = 'toself',name = "Bush"))
zradar.add_trace(go.Scatterpolar(r = clinton_z,theta= categories,fill = 'toself',name = "Clinton"))
zradar.add_trace(go.Scatterpolar(r = obama_z,theta= categories,fill = 'toself',name = "Obama"))
zradar.add_trace(go.Scatterpolar(r = trump_z,theta= categories,fill = 'toself',name = "Trump"))
zradar.update_traces(fill = "none")
zradar.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[-.5, 2]
    )),
  showlegend=True
)

zradar.show()

The above radar chart shows the relative frequency of relevant word mentions fitting each of the categories. However, more attention should be paid to the shape of each administration's trace than its size, since there is a difference in frequency of press briefings across administrations.

In [50]:
#set index of main dataframe to dates to be able to use rolling average windows
df.set_index('date',inplace=True)

In [51]:
type(df.index[0])

pandas._libs.tslibs.timestamps.Timestamp

In [67]:
time_interval = '30D'
df['r_env_z'] = df.rolling(window= time_interval,closed="right").environment_z.mean()
df['r_def_z'] = df.rolling(window=time_interval,closed="right").defense_z.mean()
df['r_econ_z'] = df.rolling(window=time_interval,closed="right").economy_z.mean()
df['r_ph_z'] = df.rolling(window=time_interval,closed="right").public_health_z.mean()
df['r_ed_z'] = df.rolling(window=time_interval,closed="right").education_z.mean()

Here, rolling 30-day averages of each category's z-score are created so that frequency of topical mentions can be tracked more smoothly over time.

In [69]:
df.head(5)

,title,text,tokenized_text,environment_count,defense_count,economy_count,public_health_count,education_count,environment_z,defense_z,economy_z,public_health_z,education_z,admin,r_env_z,r_def_z,r_econ_z,r_ph_z,r_ed_z
date,,,,,,,,,,,,,,,,,,,
2021-06-08 10:19:00,Press Briefing by White House COVID-19 Respons...,about search via teleconference 10:19 a.m. ed...,"[about, search, via, teleconference, 10:19, a....",0,3,4,14,4,-0.407324,-0.713793,-0.566748,0.923794,0.066124,Biden,-0.407324,-0.713793,-0.566748,0.923794,0.066124
2021-06-07 12:42:00,Press Briefing by Press Secretary Jen Psaki an...,about search white house press secretary jame...,"[about, search, white, house, press, secretary...",14,26,21,21,1,1.827429,0.474526,0.487367,1.644607,-0.264196,Biden,0.710052,-0.119634,-0.039690,1.284201,-0.099036
2021-06-04 13:39:00,Press Briefing by Press Secretary Jen Psaki an...,about search white house press secretary jame...,"[about, search, white, house, press, secretary...",3,13,84,14,11,0.071551,-0.197133,4.393796,0.923794,0.836869,Biden,0.497219,-0.145467,1.438138,1.164065,0.212932
2021-06-03 12:56:00,Press Briefing by Press Secretary Jen Psaki | ...,about search white house press secretary jame...,"[about, search, white, house, press, secretary...",1,18,35,22,2,-0.247699,0.061197,1.355463,1.747580,-0.154089,Biden,0.310989,-0.093801,1.417469,1.309944,0.121177
2021-06-03 11:10:00,Press Briefing by White House COVID-19 Respons...,about search via teleconference 11:10 a.m. ed...,"[about, search, via, teleconference, 11:10, a....",0,6,1,35,1,-0.407324,-0.558795,-0.752769,3.086233,-0.264196,Biden,0.167327,-0.186799,0.983422,1.665202,0.044102


In [70]:
rolling_avgs_graph = go.Figure()
rolling_avgs_graph.add_trace(go.Scatter(x=df.index,y = df['r_env_z'], name = "Environment"))
rolling_avgs_graph.add_trace(go.Scatter(x=df.index,y = df['r_def_z'], name = "Defense"))
rolling_avgs_graph.add_trace(go.Scatter(x=df.index,y = df['r_econ_z'], name = "Economy"))
rolling_avgs_graph.add_trace(go.Scatter(x=df.index,y = df['r_ph_z'], name = "Public Health"))
rolling_avgs_graph.add_trace(go.Scatter(x=df.index,y = df['r_ed_z'], name = "Education"))